## Training CNN on Multiple Machines

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer, Reshape, Rescaling
from keras.layers import Flatten, Dense
import tensorflow as tf
import json
import os

# Force using CPU only by making GPU invisible
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# TF_CONFIG envrionment variable declaration
os.environ["TF_CONFIG"] = json.dumps({
    'cluster': {
        'worker': ["192.168.10.1:12345", "192.168.10.2:12345"]
    },
    'task': {'type': 'worker', 'index': 1}
})

# Declaring Strategy before insering Data and creating Model
strategy = tf.distribute.MultiWorkerMirroredStrategy()

In [ ]:
# Insert Dataset from Hard-Drive
import tensorflow as tf

batch_size = 32
img_height = 128
img_width = 128
train_dir = ".\\rps_dataset\\rps"
test_dir = ".\\rps_dataset\\rps-test-set"
valid_dir = ".\\rps_dataset\\rps-validation"

# Training Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  subset=None,
  interpolation="bilinear",
  follow_links=False,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Test Dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  subset=None,
  interpolation="bilinear",
  follow_links=False,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Validation Dataset
valid_ds = tf.keras.utils.image_dataset_from_directory(
  valid_dir,
  subset=None,
  interpolation="bilinear",
  follow_links=False,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# Create and Compile Model
with strategy.scope():
    myCNN = Sequential()
    myCNN.add(Rescaling(1./255))
    myCNN.add(InputLayer(input_shape=(128, 128, 3)))
    myCNN.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    myCNN.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    myCNN.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    myCNN.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    myCNN.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    myCNN.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    myCNN.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    myCNN.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    myCNN.add(Flatten())
    myCNN.add(Dense(128, activation='relu'))
    myCNN.add(Dense(128, activation='relu'))
    myCNN.add(Dense(128, activation='relu'))
    myCNN.add(Dense(3, activation='softmax'))
    myCNN.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
# Training and Evaluating Model
myCNN.fit(train_ds, validation_data = valid_ds, epochs = 5)
myCNN.evaluate(test_ds)